In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from fastai.text import *
from pathlib import Path

In [3]:
base_path = Path('..')
path = base_path/'judgements.csv'
BATCH_SIZE = 16

In [4]:
df = pd.read_csv(path, encoding='latin-1')

In [5]:
df.head()

,text
0,http://JUDIS.NIC.IN SUPREME COURT OF INDIA Pag...
1,http://JUDIS.NIC.IN SUPREME COURT OF INDIA Pag...
2,http://JUDIS.NIC.IN SUPREME COURT OF INDIA Pag...
3,http://JUDIS.NIC.IN SUPREME COURT OF INDIA Pag...
4,http://JUDIS.NIC.IN SUPREME COURT OF INDIA Pag...


### Extracting only Judgements

In [6]:
df['text'] = df['text'].str.partition('JUDGMENT')[2]

In [7]:
df['text'].head()

0    :\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...
1    :\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...
2    :\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...
3    :\n21/12/1950\n\nBENCH:\nFAZAL ALI, SAIYID\nBE...
4    :\n20/12/1950\n\nBENCH:\nFAZAL ALI, SAIYID\nBE...
Name: text, dtype: object

In [8]:
df['label'] = [0]*df.shape[0]

In [9]:
df.head()

,text,label
0,":\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...",0
1,":\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...",0
2,":\n21/12/1950\n\nBENCH:\nKANIA, HIRALAL J. (CJ...",0
3,":\n21/12/1950\n\nBENCH:\nFAZAL ALI, SAIYID\nBE...",0
4,":\n20/12/1950\n\nBENCH:\nFAZAL ALI, SAIYID\nBE...",0


In [10]:
df.dropna(axis=1, how='any', inplace=True)

In [11]:
df.to_csv(base_path/'intermediate.csv', index=False)

### Text Language Model

In [12]:
data_lm = (TextLMDataBunch.from_csv(base_path, 'intermediate.csv')
           .split_by_rand_pct(0.1)
           .label_for_lm()
           .databunch(bs=BATCH_SIZE))

In [13]:
data_lm.save('data_lm.pkl')

In [14]:
data_lm = load_data(base_path, 'data_lm.pkl', bs=BATCH_SIZE)

## Language Model Learner

In [15]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time
